In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (accuracy_score, confusion_matrix, roc_curve, auc, classification_report, 
ConfusionMatrixDisplay, RocCurveDisplay, precision_recall_curve, PrecisionRecallDisplay, PredictionErrorDisplay, make_scorer)
from sklearn.utils.discovery import all_displays
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import LearningCurveDisplay, learning_curve
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.ensemble import RandomForestClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from pyrcn.echo_state_network import ESNClassifier

from scipy.stats import fisher_exact

from keras.models import Sequential
from keras.layers import LSTM, Dense

import torch

import json

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
np.random.seed(8090)

In [3]:
data = pd.read_csv('/Users/michael/Documents/MA Stats/STAT 8090/final dataset/full dataset.csv')
data['ipo_date'] = pd.to_datetime(data['ipo_date'])
data = data.sort_values(by='ipo_date')
data.head()

,Symbol,Company_Name,Exchange,Orig_Range_Low,Orig_Range_High,Price,Shares,Offer_Amount,ipo_date,IPO_Year,...,plustwo_volume_pct,plusthree_change_pct,plusthree_volume_pct,plus90_change_pct,plus90_volume_pct,plus180_change_pct,plus180_volume_pct,d1_d180,d1_pop,d1_d180_pre_price
772,BOX,Box,NYSE,11.0,13.0,14.0,12500000,175000000,2015-01-23,2015,...,0.403792,-0.055276,0.273560,-0.000547,0.045112,-0.013056,0.234416,0,1,1
771,ASND,Ascendis Pharma,Nasdaq,16.0,18.0,18.0,6000000,108000000,2015-01-28,2015,...,0.024383,-0.002626,0.011633,-0.061760,0.021800,-0.058990,0.044250,1,1,1
769,SHAK,Shake Shack,NYSE,17.0,19.0,21.0,5000000,105000000,2015-01-30,2015,...,0.273360,0.020828,0.144520,0.003064,0.103760,0.142043,0.274520,1,1,1
770,ONCE,Spark Therapeutics,Nasdaq,19.0,21.0,23.0,7000000,161000000,2015-01-30,2015,...,0.039350,-0.003979,0.028890,-0.037964,0.039806,0.004957,0.091324,1,0,1
768,NVTA,InVitae,NYSE,13.0,15.0,16.0,6350000,101600000,2015-02-12,2015,...,0.037796,0.047458,0.043847,-0.000803,0.031970,0.011261,0.053750,0,1,1


In [4]:
# validation = pd.read_csv('/Users/michael/Documents/MA Stats/STAT 8090/final dataset/validation_set.csv')

In [5]:
with open('/Users/michael/Documents/MA Stats/STAT 8090/Code Files/exchange.json','r') as f:
    exchange = json.load(f)

In [6]:
with open('/Users/michael/Documents/MA Stats/STAT 8090/Code Files/sector.json','r') as f:
    sector = json.load(f)

In [7]:
with open('/Users/michael/Documents/MA Stats/STAT 8090/Code Files/industry.json','r') as f:
    industry = json.load(f)

In [8]:
data['Exchange'] = data['Exchange'].replace(exchange)
data['Sector'] = data['Sector'].replace(sector)
data['Industry'] = data['Industry'].replace(industry)

In [9]:
# validation['Exchange'] = validation['Exchange'].replace(exchange)
# validation['Sector'] = validation['Sector'].replace(sector)
# validation['Industry'] = validation['Industry'].replace(industry)

In [10]:
data = data.drop(columns=['Symbol', 'Company_Name', 'ipo_date', 'Country_HQ', 'Pitchbook_Number',
                          'Orig_Range_Low', 'Orig_Range_High', 'Year_Founded', 'Raised_to_IPO',
                          'ipo_date_open','ipo_date_close','ipo_date_volume','plusone_open','plusone_close',
                          'plusone_volume','plustwo_open','plustwo_close','plustwo_volume','plusthree_open','plusthree_close',
                          'plusthree_volume','plus90_open','plus90_close','plus90_volume','plus180_open','plus180_close',
                          'plus180_volume','day_one_bump','ipo_date_change_pct','ipo_date_volume_pct','plusone_change_pct',
                          'plusone_volume_pct','plustwo_change_pct','plustwo_volume_pct','plusthree_change_pct',
                          'plusthree_volume_pct','plus90_change_pct','plus90_volume_pct','plus180_change_pct','plus180_volume_pct',
                          'd1_d180', 'd1_d180_pre_price'])

In [11]:
# validation = validation.drop(columns=['Symbol', 'Company_Name', 'ipo_date', 'Country_HQ', 'Pitchbook_Number',
#                           'Orig_Range_Low', 'Orig_Range_High', 'Year_Founded', 'Raised_to_IPO',
#                           'ipo_date_open','ipo_date_close','ipo_date_volume','plusone_open','plusone_close',
#                           'plusone_volume','plustwo_open','plustwo_close','plustwo_volume','plusthree_open','plusthree_close',
#                           'plusthree_volume','plus90_open','plus90_close','plus90_volume','plus180_open','plus180_close',
#                           'plus180_volume','day_one_bump','ipo_date_change_pct','ipo_date_volume_pct','plusone_change_pct',
#                           'plusone_volume_pct','plustwo_change_pct','plustwo_volume_pct','plusthree_change_pct',
#                           'plusthree_volume_pct','plus90_change_pct','plus90_volume_pct','plus180_change_pct','plus180_volume_pct',
#                           'd1_d180', 'd1_d180_pre_price'])

In [12]:
data.head()

,Exchange,Price,Shares,Offer_Amount,IPO_Year,Years_to_IPO,Price_Low_Delta,Price_High_Delta,Sector,Industry,...,market_min5,market_min4,market_min3,market_min2,market_min1,market_min0,sent_negative,sent_neutral,sent_positive,d1_pop
772,0,14.0,12500000,175000000,2015,10,0.272727,0.076923,8,136,...,10840.400000,10926.733330,10471.36522,10911.726320,10818.236360,10679.23500,5,15,0,1
771,1,18.0,6000000,108000000,2015,8,0.125000,0.000000,5,65,...,4464.830476,4551.584762,4403.23000,4687.702632,4732.700909,4673.70150,0,0,0,1
769,0,21.0,5000000,105000000,2015,11,0.235294,0.105263,1,26,...,10840.400000,10926.733330,10471.36522,10911.726320,10818.236360,10679.23500,1,1,0,1
770,1,23.0,7000000,161000000,2015,2,0.210526,0.095238,5,65,...,4464.830476,4551.584762,4403.23000,4687.702632,4732.700909,4673.70150,0,0,0,0
768,0,16.0,6350000,101600000,2015,5,0.230769,0.066667,5,66,...,10926.733330,10471.365220,10911.72632,10818.236360,10679.235000,10969.93684,0,0,0,1


#### Echo State Network

In [13]:
X = data.drop(columns=['d1_pop'])
y = data['d1_pop']

split_index = int(0.7 * len(data))

X['Exchange'] = X['Exchange'].astype('int')
X['Sector'] = X['Sector'].astype('int')
X['Industry'] = X['Industry'].astype('int')

X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8090)

# under_sampler = RandomUnderSampler()

# X_train, y_train = under_sampler.fit_resample(X_train, y_train)

# unique_values = set(X_train['Industry'])

# # Check if at least one sample from each unique value is present in the training set
# for value in unique_values:
#     if value not in X_train['Industry'].unique():
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8090)
#         break

In [14]:
# X_valid = validation.drop(columns=['d1_pop'])
# y_valid = validation['d1_pop']

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid to search
param_grid = {
    'n_reservoir': [1,2,3,4,5],  # Number of reservoir units
    'spectral_radius': [0.45, 0.5, 0.55, 0.6],  # Spectral radius
    'leakage': [0.9, 0.95, 1.0],  # Leakage rate
    'solver': ['lsqr', 'ridge'],  # Solver for linear regression
    'alpha': [0.8, 0.85, 0.9, 0.95, 1.0]  # Ridge regression parameter
}

# Initialize ESN classifier
esn = ESNClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=esn, param_grid=param_grid, cv=10, scoring='accuracy', verbose=1)
grid_search.fit(X_train_scaled, y_train)

In [ ]:
ranks = pd.DataFrame(
    {
        'Model': grid_search.cv_results_['params'],
        'Mean Test Score': grid_search.cv_results_['mean_test_score'],
        'Std Test Score': grid_search.cv_results_['std_test_score'],
        'Rank': grid_search.cv_results_['rank_test_score']
    }
)

pd.concat([ranks.drop(['Model'], axis=1), ranks['Model'].apply(pd.Series)], axis=1).sort_values(by="Rank")

In [ ]:
print("Best Parameters:", grid_search.best_params_)

In [ ]:
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_test_scaled)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
cm

In [ ]:
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
TP = cm[1, 1]

odds_ratio, p_value = fisher_exact([[TP, FP], [FN, TN]])

log_odds_ratio = np.log(odds_ratio)

std_error_log_odds_ratio = np.sqrt(1/TP + 1/FP + 1/FN + 1/TN)

z_score = 1.96  # 95% confidence interval
lower_bound = log_odds_ratio - z_score * std_error_log_odds_ratio
upper_bound = log_odds_ratio + z_score * std_error_log_odds_ratio


ci_lower = np.exp(lower_bound)
ci_upper = np.exp(upper_bound)

print("95% Confidence Interval for Odds Ratio:", (ci_lower, ci_upper))
print("p-value for Odds Ratio:", p_value)

In [ ]:
report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

In [ ]:
class_labels = ['Class 0', 'Class 1']

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.title('All Variables', fontsize=16)
plt.savefig('/Users/michael/Documents/MA Stats/STAT 8090/cm_reservoir_1.png', dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, best_estimator.predict_proba(X_test_scaled))
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('All Variables', fontsize=16)
plt.legend(loc="lower right")
plt.savefig('/Users/michael/Documents/MA Stats/STAT 8090/roc_reservoir_1.png', dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
precision, recall, _ = precision_recall_curve(y_test, best_estimator.predict_proba(X_test_scaled))

plt.figure()
plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('All Variables', fontsize=16)
plt.savefig('/Users/michael/Documents/MA Stats/STAT 8090/prc_reservoir_1.png', dpi=300, bbox_inches="tight")
plt.show()

## Feature Importance Models

### Reservior Computing

In [ ]:
model = RandomForestClassifier()
model.fit(X, y)

feature_importance = pd.DataFrame(model.feature_importances_,
                                   index=X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)

In [ ]:
fi = feature_importance[feature_importance['importance'] >= 0.01]
fi = fi.index.tolist()

In [ ]:
fi_data = X[fi]

In [ ]:
fi_data.head()

In [ ]:
split_index = int(0.7 * len(fi_data))

X_train = fi_data.iloc[:split_index]
X_test = fi_data.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(fi_data, y, test_size=0.3, random_state=8090)

# under_sampler = RandomUnderSampler()

# X_train, y_train = under_sampler.fit_resample(X_train, y_train)

# unique_values = set(X_train['Industry'])

# # Check if at least one sample from each unique value is present in the training set
# for value in unique_values:
#     if value not in X_train['Industry'].unique():
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8090)
#         break

In [ ]:
# X_valid = X_valid[fi]

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_valid_scaled = scaler.fit_transform(X_valid)

In [ ]:
# Define the parameter grid to search
param_grid = {
    'n_reservoir': [1,2,3,4,5], 
    'spectral_radius': [0.45, 0.5, 0.55, 0.6],  
    'leakage': [0.9, 0.95, 1.0],  
    'solver': ['lsqr', 'ridge'], 
    'alpha': [0.8, 0.85, 0.9, 0.95, 1.0] 
}

# Initialize ESN classifier
esn = ESNClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=esn, param_grid=param_grid, cv=10, scoring='accuracy', verbose=1)
grid_search.fit(X_train_scaled, y_train)

In [ ]:
ranks = pd.DataFrame(
    {
        'Model': grid_search.cv_results_['params'],
        'Mean Test Score': grid_search.cv_results_['mean_test_score'],
        'Std Test Score': grid_search.cv_results_['std_test_score'],
        'Rank': grid_search.cv_results_['rank_test_score']
    }
)

pd.concat([ranks.drop(['Model'], axis=1), ranks['Model'].apply(pd.Series)], axis=1).sort_values(by="Rank")

In [ ]:
print("Best Parameters:", grid_search.best_params_)

In [ ]:
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_test_scaled)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
cm

In [ ]:
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]
TP = cm[1, 1]

odds_ratio, p_value = fisher_exact([[TP, FP], [FN, TN]])

log_odds_ratio = np.log(odds_ratio)

std_error_log_odds_ratio = np.sqrt(1/TP + 1/FP + 1/FN + 1/TN)

z_score = 1.96  # 95% confidence interval
lower_bound = log_odds_ratio - z_score * std_error_log_odds_ratio
upper_bound = log_odds_ratio + z_score * std_error_log_odds_ratio


ci_lower = np.exp(lower_bound)
ci_upper = np.exp(upper_bound)

print("95% Confidence Interval for Odds Ratio:", (ci_lower, ci_upper))
print("p-value for Odds Ratio:", p_value)

In [ ]:
report = classification_report(y_test, y_pred)

print("Classification Report:")
print(report)

In [ ]:
# y_pred = best_estimator.predict(X_valid_scaled)

In [ ]:
# y_pred

In [ ]:
class_labels = ['Class 0', 'Class 1']

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.title('Feature Importance', fontsize=16)
plt.savefig('/Users/michael/Documents/MA Stats/STAT 8090/cm_reservoir_1_fi.png', dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, best_estimator.predict_proba(X_test_scaled))
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Feature Importance', fontsize=16)
plt.legend(loc="lower right")
plt.savefig('/Users/michael/Documents/MA Stats/STAT 8090/roc_reservoir_1_fi.png', dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
precision, recall, _ = precision_recall_curve(y_test, best_estimator.predict_proba(X_test_scaled))

plt.figure()
plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Feature Importance', fontsize=16)
plt.savefig('/Users/michael/Documents/MA Stats/STAT 8090/prc_reservoir_1_fi.png', dpi=300, bbox_inches="tight")
plt.show()

---
### End of Notebook